# Create the Features and Labels Files
After calculating the lifted index, we will aggregate all the features and labels together and create one feature file and one label file.

In [1]:
import os
import glob
import pandas as pd

SILVER_GPH20S10K_PATH = '/usr/datalake/silver/igra/gph20s10k'
SILVER_LI_PATH = '/usr/datalake/silver/igra/li'
GOLD_PARQUET_PATH = '/usr/datalake/silver/igra/liftedindex_lr'

# You will need the pyarrow package to create a parquet file
#%conda install pyarrow

In [2]:
# Make sure the destiation folder exists
os.makedirs(GOLD_PARQUET_PATH, exist_ok=True)


In [3]:
def df_from_concatinated_csv(path: str, pattern: str) -> pd.DataFrame:
    """Load multiple csv files into one DataFrame"""
    
    # Get the list of files from a folder
    file_list = glob.glob(path + '/' + pattern)

    # Read each CSV file into DataFrame
    # This creates a list of dataframes
    df_list = (pd.read_csv(file) for file in file_list)

    # Pandas doesn't like to concatinate "empty" DataFrames
    df_list = [dataframe for dataframe in df_list if dataframe.shape[0] > 0]

    # Concatenate all DataFrames
    return pd.concat(df_list, ignore_index=True)

def df_merge(features: pd.DataFrame, labels: pd.DataFrame):
    """Perform inner join on raw features and labels"""

    # Inner Join on id, effectie_date, and hour
    return features \
        .merge(labels, how='inner', on=['id', 'effective_date', 'hour'])

def orchestrate() -> pd.DataFrame:
    """Load and merge the raw files"""
    df_labels = df_from_concatinated_csv(SILVER_LI_PATH, '*-data-li.csv')
    df_features = df_from_concatinated_csv(SILVER_GPH20S10K_PATH, '*-data-gph20s10k.csv')
    return df_merge(df_features, df_labels)


In [4]:
# Get a combined DataFrame of the raw feauters and labels
df = orchestrate()

In [5]:
# Sanity checks
print(f"Row count: {df.shape[0]}")
print("df.info:")
df.info()

Row count: 359040
df.info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359040 entries, 0 to 359039
Columns: 131 entries, id to li
dtypes: float64(128), int64(1), object(2)
memory usage: 358.8+ MB


In [6]:
# View statistics
df.describe()

,hour,day_num,0_gph,0_pres,0_temp,0_dp,0_u,0_v,1_gph,1_pres,...,19_dp,19_u,19_v,20_gph,20_pres,20_temp,20_dp,20_u,20_v,li
count,359040.000000,359040.000000,359040.000000,359040.000000,359040.000000,359040.000000,359040.000000,359040.000000,359040.000000,359040.000000,...,359040.000000,359040.000000,359040.000000,359040.0,359040.000000,359040.000000,359040.000000,359040.000000,359040.000000,359040.000000
mean,6.447368,-0.004734,526.331258,954.856631,13.406533,5.721887,-0.218871,0.603693,798.211643,923.948908,...,-55.188882,20.509333,-0.634633,10000.0,274.690661,-44.301622,-58.234051,21.554595,-0.491100,8.435121
std,6.127930,0.712032,375.986780,42.319421,12.474994,11.359168,2.963668,3.957543,207.241836,23.118147,...,10.223609,16.019458,16.208312,0.0,12.131103,7.099340,9.636109,16.418480,16.525885,10.490250
min,0.000000,-1.000000,198.000000,826.600000,-42.000000,-46.400000,-55.200000,-89.600000,688.100000,826.600000,...,-88.800000,-49.000000,-91.100000,10000.0,231.000000,-68.500000,-90.400000,-46.700000,-105.000000,-27.400000
25%,0.000000,-0.720000,229.000000,920.300000,4.300000,-2.900000,-2.000000,-1.700000,688.100000,919.200000,...,-62.000000,9.300000,-9.400000,10000.0,266.800000,-49.800000,-64.600000,10.000000,-9.600000,-0.200000
50%,12.000000,0.000000,357.000000,973.000000,15.000000,6.300000,-0.000000,0.500000,688.100000,933.200000,...,-55.300000,19.400000,-0.500000,10000.0,276.500000,-44.300000,-58.500000,20.500000,-0.500000,7.000000
75%,12.000000,0.710000,849.000000,987.300000,23.000000,15.400000,1.500000,2.900000,849.000000,938.300000,...,-48.000000,30.800000,8.500000,10000.0,283.700000,-38.700000,-51.500000,32.200000,8.900000,16.000000
max,23.000000,1.000000,1475.000000,1022.800000,43.700000,37.100000,38.600000,46.300000,1475.000000,964.600000,...,-5.000000,109.000000,80.400000,10000.0,520.200000,-4.800000,-6.000000,107.800000,128.600000,57.300000


In [7]:
# Write the parquet file.
df.to_parquet(GOLD_PARQUET_PATH + '/gph20s10k_li.parquet')
